# 空氣污染監測網 網路爬蟲實作練習


* 能夠利用 selenium + BeautifulSoup 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
* ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料





In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
import time

### ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料

In [2]:
# 打開瀏覽器
browser = webdriver.Chrome(executable_path='chromedriver')
browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")
# 模擬使用者操作行為，選擇/點擊
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2018')
browser.find_element_by_id('ctl05_btnQuery').click()

time.sleep(2)

# 取得資料，丟到 BeautifulSoup 解析
html_source = browser.page_source
soup = BeautifulSoup(html_source, 'html.parser')
table = soup.find('table', class_='TABLE_G')

th = table.find_all('th')
columns = [tag.text for tag in th]

d = {}
td = table.find_all('td')

for tag in td:
    if tag.attrs and tag.get('style'):
        d.setdefault(tag.text, {})
    elif not tag.attrs and tag.text.strip():
        if '/' in tag.text:
            k = tag.text
        else:
            d[list(d.keys())[-1]].setdefault(k, tag.text)

df = pd.DataFrame(d)
df2 = df[['SO2']].iloc[:8].reset_index()
df2.columns = columns[2:4]
df2

,監測日期,監測值
0,2018/01,1.80
1,2018/02,1.90
2,2018/03,2.20
3,2018/04,2.30
4,2018/05,3.10
5,2018/06,2.70
6,2018/07,2.20
7,2018/08,2.40


### ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

In [3]:
df = pd.DataFrame(d)
df2 = df[['SO2', 'CO']].iloc[:8].reset_index()
old_columns = list(df2.columns)
new_columns = columns[2:3] + \
              list(map(lambda x: str(x) + '-' + ''.join(columns[3:4]), old_columns[1:]))
df2.columns = new_columns
df2

,監測日期,SO2-監測值,CO-監測值
0,2018/01,1.80,0.34
1,2018/02,1.90,0.44
2,2018/03,2.20,0.40
3,2018/04,2.30,0.38
4,2018/05,3.10,0.34
5,2018/06,2.70,0.29
6,2018/07,2.20,0.21
7,2018/08,2.40,0.30


In [4]:
# 關閉瀏覽器
browser.quit()